### 计算从公园出发n分钟内的amentity数目

#### 1. 加载公园数据

In [5]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx

# 选择适合的投影坐标系统
projected_crs = '3857'  # Web Mercator 投影

# 读取seattle 边界
seattle_boundary = gpd.read_file("Seattle_City_Boundary.geojson")
seattle_boundary = seattle_boundary.to_crs(epsg=projected_crs) 

In [8]:
parks_detail = gpd.read_file('Park_Details.geojson')
parks_detail = parks_detail.rename(columns = {'PMA':'TOTAL_AREA','USE_':'TYPE'})
parks_detail = parks_detail.to_crs(epsg = projected_crs)
parks_detail = parks_detail[parks_detail.is_valid]
parks_detail = gpd.clip(parks_detail,seattle_boundary)
parks_detail['centroid'] = parks_detail.geometry.centroid

#### 2. 加载amentity 数据

In [10]:
def ReadAmenityData(file_path,clip_mask,projected_crs):
    
    # 读取
    amenity_gdf = gpd.read_file(file_path)

    # 转换坐标系统
    if amenity_gdf.crs != projected_crs:
        amenity_gdf = amenity_gdf.to_crs(epsg=projected_crs)

    # 裁剪
    amenity_gdf_clip = gpd.clip(amenity_gdf,clip_mask)

    return amenity_gdf_clip

In [12]:
# 1. 读取学校的数据
schools_gdf = ReadAmenityData(file_path = 'Seattle_Public_Schools_Sites.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
schools_gdf = schools_gdf[~schools_gdf['Status'].str.contains('Closed')]
schools_gdf.head()

,OBJECTID_1,OBJECTID,schID,schName,mapLabel,Status,esmshs,geometry
2,3,3,264,Rainier View,Rainier View,ELEM,ES,POINT (-13610274.057 6023884.827)
4,5,5,221,Emerson,Emerson,ELEM,ES,POINT (-13609769.121 6026514.487)
6,7,7,291,South Shore K-8,South Shore K-8,Option ELEM,ES,POINT (-13611257.794 6027985.217)
8,9,9,21,Rainier Beach,Rainier Beach,HS,HS,POINT (-13610573.163 6028039.258)
10,11,11,219,Dunlap,Dunlap,ELEM,ES,POINT (-13611552.404 6028190.661)


In [13]:
# 2. 读取landmark的数据
landmarks_gdf = ReadAmenityData(file_path = 'Landmarks_and_Government_Buildings.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
landmarks_gdf.head()

,OBJECTID,PERMANENT_IDENTIFIER,SOURCE_FEATUREID,SOURCE_DATASETID,SOURCE_DATADESC,SOURCE_ORIGINATOR,DATA_SECURITY,DISTRIBUTION_POLICY,LOADDATE,FTYPE,...,ADDRESSBUILDINGNAME,ADDRESS,CITY,STATE,ZIPCODE,GNIS_ID,FOOT_ID,COMPLEX_ID,GLOBALID,geometry
184874,1003832,{A4D79E6D-E90F-4E40-B933-5434EEA8FB65},None,2f97153f-59ba-442e-8b8e-c222ac07af34,780 TNMC Update 2/21/2023,U.S. Geological Survey,5.0,E4,"Tue, 21 Feb 2023 15:50:41 GMT",780,...,None,2721 Southwest Trenton Street,Seattle,WA,98126,2462269,None,None,5a46ac9d-5a03-46ba-a9af-a905d86c59f2,POINT (-13621845.468 6028039.808)
17019,1051652,{6E7174F1-8105-4D8E-8AA9-F6E12E7919CA},None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,5.0,E4,"Wed, 08 Mar 2023 21:53:12 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,1504959,None,None,566da1b2-9ab4-45ba-a686-4b0e4277a25a,POINT (-13621952.689 6030937.665)
171565,1471050,317a4f56-3ecd-4ca4-b73a-2eb43826af1f,None,96d62c00-eeea-4553-9086-0b4cbfe527c6,820 TNMC Update 3/8/2023,U.S. Geological Survey,0.0,E4,"Wed, 08 Mar 2023 21:53:07 GMT",820,...,None,6701 30th Avenue Southwest,Seattle,WA,98126,None,None,None,7c50bbdd-a30a-44df-a8c6-5b0541b235bc,POINT (-13622254.81 6031017.647)
171593,1471085,9957cb1b-a289-488f-ae3e-e9f2ffd50a9e,None,52740733-6aaf-4a55-967c-eb4358ba9a6d,820 TNMC Update 1/11/2024,U.S. Geological Survey,0.0,E4,"Thu, 11 Jan 2024 03:02:47 GMT",820,...,None,22nd Avenue South,Seattle,WA,98108,None,None,None,996c8467-6c8b-4d03-a2d3-59af5d9364b6,POINT (-13614950.916 6031818.468)
184858,1003815,{A71622B7-BAA4-48B1-8CC7-D8A1DD759EE5},None,86aaf0c3-7986-4d31-a3a5-e7154a52221d,780 TNMC Update 3/11/2023,U.S. Geological Survey,5.0,E4,"Sat, 11 Mar 2023 02:46:35 GMT",780,...,None,620 South Orcas Street,Seattle,WA,98108,2462221,None,None,1a953dfe-4bec-47fd-8360-eed9eeb67d63,POINT (-13617253.003 6032669.504)


In [14]:
# 3. 读取metro stop的数据
metro_stops_gdf = ReadAmenityData(file_path = 'King_County_Metro_Transit_Stops.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
metro_stops_gdf.head()

,OBJECTID_1,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
13132,13133,13133,155,4,YES,None,55380,127316,ACT,REG,...,Beacon Ave S,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-asphlt,Single,1,4715,POINT (-13609919.788 6023459.351)
19051,19052,19052,155,5,YES,None,55380,127316,ACT,REG,...,Beacon Ave S,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-asphlt,Single,1,4715,POINT (-13609919.788 6023459.351)
13123,13124,13124,155,4,YES,None,55370,48188,ACT,REG,...,59th Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4707,POINT (-13609888.325 6023633.418)
20216,20217,20217,155,5,YES,None,55370,48188,ACT,REG,...,59th Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4707,POINT (-13609888.325 6023633.418)
843,844,844,155,4,YES,None,55430,51575,ACT,REG,...,51st Ave S,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,4720,POINT (-13611040.48 6024869.901)


In [16]:
# 4. 读取housing的数据
housing_gdf = ReadAmenityData(file_path = 'King_County_Assessor_Residential_Unit_Types_and_Sizes.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
housing_gdf = housing_gdf[housing_gdf['UNIT_TYPE'].str.contains('RES-2 Unit|RES-3 Unit|RES-4 Unit|Apartments', na=False)]
housing_gdf.rename(columns={'SQFTTOTLIVING:': 'square of living space'}, inplace=True)
housing_gdf.head()

,OBJECTID,ADDRESS,MAJOR,PIN,BLDGNBR,PRES_USE,SECTION_USE,UNIT_TYPE,NBRTHISTYPE,BEDROOMS,...,VILLNUMB,UV_NAME,UC_NAME,CRA_NAME,TRBL20,POINT_X,POINT_Y,LAT,LON,geometry
159065,159066,5542 S 119TH ST,334840,3348401367,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3014,1.287169e+06,184485.440210,47.496766,-122.262382,POINT (-13610184.472 6023538.253)
159060,159061,5548 S 119TH ST,334840,3348401365,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3014,1.287236e+06,184485.083255,47.496768,-122.262113,POINT (-13610154.515 6023538.649)
159068,159069,11650 54TH AVE S,334840,3348401416,1,Duplex,Residence (348),RES-2 Unit,2,3,...,0.0,Outside Villages,None,Rainier Beach,11901.3008,1.286355e+06,184860.328896,47.497752,-122.265702,POINT (-13610554.107 6023700.727)
159064,159065,5160 S AUGUSTA ST,334840,3348401584,1,Duplex,Residence (348),RES-2 Unit,2,4,...,0.0,Outside Villages,None,Rainier Beach,11901.3004,1.285813e+06,185644.090451,47.499872,-122.267955,POINT (-13610804.885 6024050.138)
159199,159200,5154 S AUGUSTA ST,334840,3348401585,1,Duplex,Residence (348),RES-2 Unit,2,3,...,0.0,Outside Villages,None,Rainier Beach,11901.3004,1.285750e+06,185670.215399,47.499941,-122.268211,POINT (-13610833.382 6024061.408)


In [18]:
# 5. 读取工作机会的数据
jobs_gdf = ReadAmenityData(file_path = 'Business_Points_of_Interest_Seattle1.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
jobs_gdf.rename(columns={'EMPNUM': 'Employers Number'}, inplace=True)
jobs_gdf.head()

,OBJECTID,CONAME,ADDR,CITY,STATE_NAME,STATE,ZIP,ZIP4,NAICS,NAICS_ALL,...,SOURCE,ESRI_PID,DESC_,LATITUDE,LONGITUDE,CreationDate,Creator,EditDate,Editor,geometry
1187,1188,Ideoclick Inc,1st Ave S,Seattle,Washington,WA,98104,4420,51821013,51821013,...,Data Axle,853edb450c88380e121dde3faa397513,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
1329,1330,Code Systems Corp,1st Ave S,Seattle,Washington,WA,98104,4420,51321005,51321005,...,Data Axle,c0c4e438b4811963fdd9d9762d060b7f,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
926,927,Graham Construction & Engineering,1st Ave S,Seattle,Washington,WA,98104,4420,23611512,"23611512, 23611505, 23799012",...,Data Axle,c7559040fecc57731ecb483b0126b15f,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
3941,3942,Sice Agencia Chile Sa,Occidental Ave S,Seattle,Washington,WA,98104,2869,99999004,99999004,...,Data Axle,3fe875f7c4ccdab459c1890f23cc8dcc,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)
1911,1912,Five Six Eight First Ave,1st Ave S,Seattle,Washington,WA,98104,4419,99999004,99999004,...,Data Axle,814e6019f4011b4684243e8021abe270,"CONAME, Seattle, Washington",47.596311,-122.333688,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,"Tue, 12 Dec 2023 12:49:55 GMT",ULI_Hines_Admin_Support,POINT (-13618123.855 6039956.219)


In [20]:
# 6. 读取MHA数据（affordable housing）
MHA_gdf= ReadAmenityData(file_path = 'Seattle_Mandatory_Housing_Affordability_MHA_Zones.geojson',clip_mask = seattle_boundary,projected_crs = '3857')
MHA_gdf['centroid'] = MHA_gdf.geometry.centroid
MHA_gdf.head()

,OBJECTID,ZONEID,ZONING,CONTRACT,ORDINANCE,EFFECTIVE,HISTORIC,PEDESTRIAN,SHORELINE,OVERLAY,...,PEDESTRIAN_PREV,SHORELINE_PREV,OVERLAY_PREV,LIGHTRAIL_PREV,BZONEID,PUBLIC_DESCRIPTION,CHAPTER,CHAPTER_LINK,geometry,centroid
671,672,5224,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13610031.832 6023559.014, -13610013...",POINT (-13609944.022 6023467.407)
691,692,5252,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608806.519 6024176.479, -13608824...",POINT (-13608748.347 6024221.13)
1677,1678,4957,LR1 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,2966.0,is a multifamily residential zone where reside...,Chapter 23.45,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13611047.866 6025230.025, -13610974...",POINT (-13611012.153 6025175.461)
1379,1380,4708,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13608555.044 6025896.358, -13608555...",POINT (-13608593.266 6025912.224)
2030,2031,4714,NC1-40 (M),,125791,"Fri, 19 Apr 2019 00:00:00 GMT",,,,,...,,,,,4135.0,is a mixed-use zone where both residential and...,Chapter 23.47A,https://library.municode.com/wa/seattle/codes/...,"POLYGON ((-13609328.712 6026711.971, -13609324...",POINT (-13609354.246 6026733.578)


#### 3. 计算xx分钟内可达的xx数目

##### 3.1 获取等时线矩阵

In [21]:
# 获取等时线矩阵
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import numpy as np
import time
import requests

# mapbox
def GetIsochroneMapbox(centroid,profile,minutes,access_token):
    lon,lat = centroid.x,centroid.y
    url = f"https://api.mapbox.com/isochrone/v1/mapbox/{profile}/{lon},{lat}?contours_minutes={minutes}&access_token={access_token}"
    retries = 3
    while retries > 0:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'features' in data and len(data['features']) > 0:
                coordinates = data['features'][0]['geometry']['coordinates']
                polygon = Polygon(coordinates)
                return polygon
            else:
                print(f"No features found for location: ({lon}, {lat})")
                return None
        elif response.status_code == 429:
            print('Rate limit exceeded. Retrying...')
            time.sleep(60)
            retries -= 1
        else:
            print(f"Failed to fetch isochrone data for location: ({lon}, {lat})")
            return None
        time.sleep(12)  # 每次请求后等待12秒（相当于每分钟5次请求）
    return None

In [23]:
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point

# 转换函数
def transform_point(_transformer,point):
    lon, lat = _transformer.transform(point.x, point.y)
    return Point(lon, lat)
    
# 获取中心点
parks_centroids = parks_detail.geometry.centroid

profile='walking'
travel_time = 10
# 获取等时线
accesstoken = 'pk.eyJ1Ijoiemh1b2FueXVhbnl1IiwiYSI6ImNtZ2htNXFmcTBoNGEycm9lbTY1MDZpYzMifQ.9XG1lOyIj4q5W5xNA3fw8g'
isochrones =[]
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
# 对于每一行获取等时矩阵
for index,row in parks_detail.iterrows():
    centroid = transform_point(transformer, row.geometry.centroid)
    isochrone = GetIsochroneMapbox(centroid, profile = profile,minutes = travel_time, access_token=accesstoken)
    isochrones.append(isochrone)

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

作用
Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True) 的作用是创建一个坐标转换对象，用于将地理坐标从 EPSG:3857 坐标系转换到 EPSG:4326 坐标系。EPSG:3857 是 Web 墨卡托投影坐标系，常用于网络地图服务（如 Google Maps、OpenStreetMap）；EPSG:4326 是 WGS84 地理坐标系，以经度和纬度表示地理位置，是全球通用的地理坐标系统。通过这个转换对象，可以将地图服务中使用的投影坐标转换为通用的经纬度坐标，方便进行地理数据的分析和处理。

原理
该方法基于 pyproj 库实现，Transformer.from_crs 方法会根据输入的两个 EPSG 代码，查找对应的投影参数和转换公式。EPSG 是欧洲石油调查组织（European Petroleum Survey Group）定义的坐标参考系统代码，每个代码对应一种特定的坐标系。Transformer 对象会根据这些代码，调用底层的投影转换算法，将输入的 EPSG:3857 坐标转换为 EPSG:4326 坐标。always_xy=True 参数确保输入和输出的坐标顺序为 (x, y) 或 (经度, 纬度)，避免因不同库对坐标顺序的不同约定而产生混淆。

**正确的理解顺序**

from pyproj import Transformer

import geopandas as gpd

from shapely.geometry import Point

**定义转换点的坐标系统**

def transform_point(_transformer,point):

    lon, lat = _transformer.transform(point.x, point.y)
    
    return Point(lon, lat)
    
parks_centroids = parks_detail.geometry.centroid

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

**对于每一行获取等时矩阵**

profile='walking'

travel_time = 10

accesstoken = 'pk.eyJ1Ijoiemh1b2FueXVhbnl1IiwiYSI6ImNtZ2htNXFmcTBoNGEycm9lbTY1MDZpYzMifQ.9XG1lOyIj4q5W5xNA3fw8g'

isochrones =[]

for index,row in parks_detail.iterrows():

    centroid = transform_point(transformer, row.geometry.centroid)
    
    isochrone = GetIsochroneMapbox(centroid, profile = profile,minutes = travel_time, access_token=accesstoken)
    
    isochrones.append(isochrone)

In [27]:
# 创建GDF,并把等时线作为geometry列
isochrones_gdf = gpd.GeoDataFrame({'geometry':isochrones})

# 坐标转换
isochrones_gdf.set_crs(epsg = 4326,inplace = True,allow_override = True)
# 在进行空间分析之前，确保所有涉及的 GeoDataFrame 使用相同的 CRS，一定要用wgs84

# 转换crs
isochrones_gdf_proj = isochrones_gdf.to_crs(epsg = projected_crs)

# 给每个公园点匹配等时线
parks_detail['isochrones'] = isochrones_gdf_proj['geometry']
parks_detail.set_geometry('geometry',inplace = True)
parks_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,geometry,centroid,isochrones
50,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Wed, 06 Jan 2010 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,"POLYGON ((-13622737.379 6023640.776, -13622737...",POINT (-13622744.893 6023634.084),"POLYGON ((-13623319.47 6031677.956, -13623585...."
1625,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,"POLYGON ((-13622809.153 6023663.187, -13622830...",POINT (-13622815.094 6023678.226),"POLYGON ((-13612891.839 6039350.079, -13613003..."
2591,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,"POLYGON ((-13622749.842 6023792.899, -13622739...",POINT (-13622787.82 6023732.238),"POLYGON ((-13621166.439 6062868.22, -13621376...."
992,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Wed, 07 Apr 2004 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,"POLYGON ((-13622845.404 6023793.752, -13622846...",POINT (-13622852.34 6023754.705),"POLYGON ((-13611805.027 6028358.78, -13611916...."
1523,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,"POLYGON ((-13622922.56 6023813.85, -13622922.8...",POINT (-13622926.852 6023768.189),"POLYGON ((-13616346.083 6038320.133, -13616568..."


In [28]:
# 可以存取中间项
parks_detail.to_csv(f'park_details_isochrones_{travel_time}_{profile}_gdf.csv', index=True)

In [29]:
# WKT（Well-Known Text）是一种用于表示几何对象的文本格式。
# 它是一个基于文本的格式，用于描述几何形状，例如点、线、多边形等。这种格式通常用于地理信息系统（GIS）中，作为一种标准的几何数据交换格式。
# 存储geojson时,将带有地理属性的列.to_wkt()
parks_detail['isochrones'] = parks_detail['isochrones'].to_wkt()
parks_detail['centroid'] = parks_detail['centroid'].to_wkt()
parks_detail.to_file(f'park_details_isochrones_{travel_time}_{profile}_gdf.geojson', driver='GeoJSON')

##### 3.2 在获取等时线并且合并进parks_details之后,计算xx时间范围内xx的数目

In [45]:
# 定义计算固定时间可到达设施的数目函数

def CalculateAmenityCountInIsochrones(amenity_gdf_clip, amenity_name, intersect_grid):
    
    # 执行空间连接，找到每个等时线范围内的设施
    amenity_within_isochrones = gpd.sjoin(amenity_gdf_clip,intersect_grid,how = 'inner',predicate = 'within')

    # 统计每个等时线范围内的设施数目
    amenity_counts = amenity_within_isochrones.groupby('index_right').size()

    # 创建一个新的列,填充统计结果
    amenity_index = f'{amenity_name}_count'
    intersect_grid[amenity_index] = intersect_grid.index.map(amenity_counts).fillna(0).astype(int)

    return intersect_grid

In [32]:
# 读取这个文件
park_detail= gpd.read_file('park_details_isochrones_10_walking_gdf.geojson')
park_detail.head()

,OBJECTID,NAME,PMA_NAME,ADDRESS,PIN,SUBPARCEL,TOTAL_AREA,OWNER,LEASE,MAINT,...,REVIEW_DATE,AMWOID,SDQL,SE_ANNO_CAD_DATA,GIS_CRT_DT,GIS_EDT_DT,GlobalID,centroid,isochrones,geometry
0,660345,SEOLA PARK,Seola Park,,7110000066,22024.0,466,DPR,N,DPR,...,"Wed, 06 Jan 2010 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",256c97cb-6578-415f-b9fe-853a0b214545,POINT (-13622744.893154 6023634.083662),"POLYGON ((-13623319.469617 6031677.955953, -13...","POLYGON ((-13622737.379 6023640.776, -13622737..."
1,659362,SEOLA PARK,Seola Park,,1123039030,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",4ebb3649-92bc-47a2-ac6a-1b390060f5d7,POINT (-13622815.0938 6023678.225553),"POLYGON ((-13612891.838956 6039350.07867, -136...","POLYGON ((-13622809.153 6023663.187, -13622830..."
2,659360,SEOLA PARK,Seola Park,,1223039013,9627.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",b7f6f0f5-97ac-4e35-ab5a-51e612165b99,POINT (-13622787.819986 6023732.238333),"POLYGON ((-13621166.439346 6062868.220189, -13...","POLYGON ((-13622749.842 6023792.899, -13622739..."
3,660174,SEOLA PARK,Seola Park,None,None,9627.0,466,DPR,N,DPR,...,"Wed, 07 Apr 2004 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",e1df0af1-cdbb-463c-b3f0-a0da44649b78,POINT (-13622852.340365 6023754.704958),"POLYGON ((-13611805.026767 6028358.780275, -13...","POLYGON ((-13622845.404 6023793.752, -13622846..."
4,659359,SEOLA PARK,Seola Park,,1123039028,9628.0,466,DPR,N,DPR,...,"Wed, 06 May 1998 00:00:00 GMT",PROPERTY-SEOPK,QL-D1,,"Mon, 12 Aug 2024 10:38:53 GMT","Mon, 12 Aug 2024 10:38:53 GMT",bb2b9b5f-1341-43a0-87f8-114467925da4,POINT (-13622926.851761 6023768.188662),"POLYGON ((-13616346.082755 6038320.133472, -13...","POLYGON ((-13622922.56 6023813.85, -13622922.8..."


In [33]:
# 读取等时矩阵
from shapely import wkt
# 第一次读取的时候
parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)
parks_detail['centroid'] = parks_detail['centroid'].apply(wkt.loads)

In [34]:
# Create GeoDataFrame
selected_columns = parks_detail[['OBJECTID', 'isochrones','centroid']]
parks_isochrones = gpd.GeoDataFrame(selected_columns, geometry='isochrones')
parks_isochrones.set_crs(epsg=projected_crs,inplace=True)
parks_isochrones = parks_isochrones.to_crs(epsg=projected_crs)
parks_isochrones.head()

,OBJECTID,isochrones,centroid
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662)
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553)
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333)
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958)
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662)


In [46]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = schools_gdf,amenity_name = 'Schools',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0


In [48]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = landmarks_gdf,amenity_name = 'Landmarks',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0


In [49]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = metro_stops_gdf,amenity_name = 'Metros',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34


In [51]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = housing_gdf,amenity_name = 'housing_gdf',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24


In [52]:
parks_isochrones = CalculateAmenityCountInIsochrones(amenity_gdf_clip = MHA_gdf,amenity_name = 'affordalbe_housings_gdf',intersect_grid = parks_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count
50,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6
1625,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0
2591,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0
992,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34
1523,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20


In [59]:
# 因为每个工作点 对应了多个工作机会。
# 执行空间连接
jobs_within_isochrones = gpd.sjoin(jobs_gdf, parks_isochrones, how='inner', predicate='within')

# 计算每个区域内 Employers Number 的总和
aggregated_employers = jobs_within_isochrones.groupby('index_right')['Employers Number'].sum().reset_index()

# 将总和信息添加到 parks_detail
parks_isochrones = parks_isochrones.merge(aggregated_employers, left_index=True, right_on='index_right', how='left')

# 重命名新列
parks_isochrones = parks_isochrones.rename(columns={'Employers Number': 'job_count'})

# 填充缺失值，并确保列的类型为整数
parks_isochrones['job_count'] = parks_isochrones['job_count'].fillna(0).astype(int)

# 删掉index_right这一列
parks_isochrones = parks_isochrones.drop(columns=['index_right'])

parks_isochrones.head()
parks_isochrones.to_csv(f'park_details_{profile}_{travel_time}_Amentities.csv')

### 计算从公园出发到最近的xx的距离

筛选我们有的数据点 | 10分钟范围内

计算距离

在 Python 中，for _, park in parks.iterrows() 是用于遍历 Pandas DataFrame parks 每一行的循环结构。parks.iterrows() 是 DataFrame 对象的一个方法，它会返回一个迭代器，该迭代器会逐行遍历 DataFrame。每次迭代会返回一个包含两个元素的元组，第一个元素是行索引，第二个元素是包含该行数据的 Series 对象。

在这个代码中，_ 是一个惯用的占位符变量名，用于表示在当前循环中不需要使用的变量。这里 _ 代表行索引，而 park 则代表包含该行数据的 Series 对象。通过这种方式，可以直接访问每一行的数据而忽略行索引。

In [62]:
# 计算每个公园到最近amenity的距离/米
def CalculateNearestAmenityDistance(parks,amenity_within_isochrones):
    distance = []
    for _,park in parks.iterrows():
        # 公园中心点
        park_point = park.geometry.centroid
        # 计算公园到筛选后设施的距离
        distances_to_amenity = amenity_within_isochrones.geometry.distance(park_point)
        # 找到最小距离
        nearest_distance = distances_to_amenity.min() if not distances_to_amenity.empty else float('inf')
        # not distances_to_amenity.empty：检查 distances_to_amenity 是否为空。distances_to_amenity 可能是一个包含到便利设施距离的数组、列表或者 Pandas 的 Series 对象。
        # 如果 distances_to_amenity 不为空（即 not distances_to_amenity.empty 为 True），则执行 distances_to_amenity.min()。这会返回 distances_to_amenity 中的最小值，也就是最近的距离。
        # 如果 distances_to_amenity 为空（即 not distances_to_amenity.empty 为 False），则将 nearest_distance 赋值为 float('inf')，表示无穷大。这意味着没有找到任何距离数据，所以最近距离被认为是无穷远。
        distance.append(nearest_distance)
    return distance

In [63]:
amenity_within_isochrones = gpd.sjoin(metro_stops_gdf,parks_isochrones,how = 'inner',predicate = 'within')
parks_isochrones['nearest_metro_distance'] = CalculateNearestAmenityDistance(parks_detail,amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance
NaN,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6,0,0,419.859360
NaN,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0,0,0,342.327526
NaN,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0,0,0,317.948334
NaN,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34,0,0,258.576351
NaN,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20,0,0,208.716468


In [65]:
amenity_within_isochrones = gpd.sjoin(landmarks_gdf,parks_isochrones,how = 'inner',predicate = 'within')
parks_isochrones['nearest_landmark_distance'] = CalculateNearestAmenityDistance(parks_detail,amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance,nearest_landmark_distance
NaN,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6,0,0,419.859360,4496.596161
NaN,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0,0,0,342.327526,4468.062235
NaN,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0,0,0,317.948334,4409.443064
NaN,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34,0,0,258.576351,4401.806958
NaN,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20,0,0,208.716468,4406.373465


In [66]:
amenity_within_isochrones = gpd.sjoin(housing_gdf,parks_isochrones,how = 'inner',predicate = 'within')
parks_isochrones['nearest_residential_distance'] = CalculateNearestAmenityDistance(parks_detail,amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance,nearest_landmark_distance,nearest_residential_distance
NaN,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6,0,0,419.859360,4496.596161,145.193647
NaN,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0,0,0,342.327526,4468.062235,73.560633
NaN,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0,0,0,317.948334,4409.443064,120.894888
NaN,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34,0,0,258.576351,4401.806958,97.815151
NaN,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20,0,0,208.716468,4406.373465,112.178215


In [67]:
amenity_within_isochrones = gpd.sjoin(MHA_gdf,parks_isochrones,how = 'inner',predicate = 'within')
parks_isochrones['nearest_MHA_distance'] = CalculateNearestAmenityDistance(parks_detail,amenity_within_isochrones)
parks_isochrones.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance,nearest_landmark_distance,nearest_residential_distance,nearest_MHA_distance
NaN,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6,0,0,419.859360,4496.596161,145.193647,787.227439
NaN,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0,0,0,342.327526,4468.062235,73.560633,736.856488
NaN,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0,0,0,317.948334,4409.443064,120.894888,684.849126
NaN,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34,0,0,258.576351,4401.806958,97.815151,659.453271
NaN,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20,0,0,208.716468,4406.373465,112.178215,646.554081


In [70]:
parks_isochrones_copy = parks_isochrones.reset_index().drop('index',axis = 1)
parks_isochrones_copy.head()

,OBJECTID,isochrones,centroid,Schools_count,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance,nearest_landmark_distance,nearest_residential_distance,nearest_MHA_distance
0,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1,0,44,16,6,0,0,419.859360,4496.596161,145.193647,787.227439
1,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),0,0,16,0,0,0,0,342.327526,4468.062235,73.560633,736.856488
2,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),0,0,8,0,0,0,0,317.948334,4409.443064,120.894888,684.849126
3,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),0,0,78,22,34,0,0,258.576351,4401.806958,97.815151,659.453271
4,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),0,0,34,24,20,0,0,208.716468,4406.373465,112.178215,646.554081


In [71]:
parks_isochrones = parks_isochrones_copy

In [72]:
parks_isochrones.to_csv(f'park_deatails_{profile}_{travel_time}_Distance_Amentities.csv',index=True)

### osmnx 计算每个公园到最近自行车网络的距离

In [74]:
# 1.加载自行车网络
import numpy as np
import osmnx as ox

# 使用 osmnx 库从 GrapghML 文件加载自行车网络数据
bike_network = ox.load_graphml('bike_network.graphml')
nodes,edges = ox.graph_to_gdfs(bike_network)

# 重置索引,将'u''v'转化为普通列
edges = edges.reset_index()

In [80]:
# 2.计算每个公园到所有自行车网络边的距离
def CalculateNearestDistance(point,edges):
    distances = [point.distance(edge) for edge in edges.geometry]
    return min(distances) if distances else float('inf')

edges = edges.to_crs(epsg=projected_crs)
edges_within_isochrones = gpd.sjoin(edges, parks_isochrones, how='inner', predicate='within')
edges_within_isochrones.head()

,u,v,key,osmid,oneway,lanes,highway,junction,reversed,length,...,Landmarks_count,Metros_count,housing_gdf_count,affordalbe_housings_gdf_count,job_count,job_count,nearest_metro_distance,nearest_landmark_distance,nearest_residential_distance,nearest_MHA_distance
0,29449863,29464223,0,426250827,True,2,secondary,intersection,False,13.474,...,0,42,7,4,0,0,211.324917,2252.155928,210.103823,144.893047
0,29449863,29464223,0,426250827,True,2,secondary,intersection,False,13.474,...,0,42,7,4,0,0,394.727822,2184.207711,90.787486,0.000000
0,29449863,29464223,0,426250827,True,2,secondary,intersection,False,13.474,...,0,42,7,4,0,0,334.931358,4494.882006,277.558745,0.000000
0,29449863,29464223,0,426250827,True,2,secondary,intersection,False,13.474,...,0,38,7,4,0,0,149.355775,3311.878033,109.691467,24.269023
0,29449863,29464223,0,426250827,True,2,secondary,intersection,False,13.474,...,0,54,5,2,0,0,383.796567,3099.657482,181.654385,120.122260


In [ ]:
parks_detail['distance_to_nearest_bike_network'] = parks_detail.geometry.centroid.apply(lambda x: CalculateNearestDistance(x, edges_within_isochrones))

In [ ]:
parks_detail.to_csv("parks_detail_accessibility_10_bike_network.csv", index = True)